In [1]:
# imports
import sys
import pandas as pd
import string
from nltk.tokenize import word_tokenize
import numpy as np
import os, sys
from gensim.models import Word2Vec
%load_ext autoreload 
%autoreload 2

In [2]:
parent_path = os.path.split(os.getcwd())[0]
if parent_path  not in sys.path:
    sys.path.append(parent_path)

In [103]:
# Imports from local module
from wots_cookin.data import load_clean_data, basic_clean, remove_formatting, remove_stopwords_from_list, load_full_stopwords
from wots_cookin.word2vec_trainer import *
from wots_cookin.shortlist import *

In [5]:
df = pd.read_pickle("../raw_data/enriched_recipes.pkl")
df.drop(columns = ['index'], inplace = True)

In [ ]:
ingredients = ['chicken', 'onion', 'spring', 'onion', 'celery', 'garlic', 'leaks']

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13487 entries, 0 to 13486
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                13487 non-null  object
 1   Ingredients          13487 non-null  object
 2   Instructions         13487 non-null  object
 3   Image_Name           13487 non-null  object
 4   Cleaned_Ingredients  13487 non-null  object
 5   Bag_Of_Ingredients   13487 non-null  object
 6   Ingredients_Length   13487 non-null  int64 
 7   Dairy free           13487 non-null  int64 
 8   No eggs              13487 non-null  int64 
 9   Nut free             13487 non-null  int64 
 10  No shellfish         13487 non-null  int64 
 11  Gluten free          13487 non-null  int64 
 12  No soy               13487 non-null  int64 
 13  Vegetarian           13487 non-null  int64 
 14  Vegan                13487 non-null  int64 
 15  Vector_List          13487 non-null  object
dtypes: i

In [7]:
df['Vector_List'][0]

array([-0.6576614 ,  0.08998758, -0.5539392 , -0.2389411 , -0.11817149,
       -0.47335452,  0.6401035 ,  0.16134764, -1.4129729 , -0.3103403 ,
       -0.33989453, -0.7942448 ,  0.3331764 , -0.02060852,  0.874237  ,
        0.71043533,  0.17645863,  0.7359437 , -0.38619316, -0.10781728,
       -0.59816855,  0.01483529,  1.1520963 ,  0.12194681,  0.21909334,
       -0.875582  ,  0.21959363,  0.32215866,  0.24488746, -0.01187852,
        0.40558362, -0.97944295,  0.22494248, -0.7216437 ,  0.0904078 ,
        0.45701835,  0.29935   , -0.09573941, -0.16305234,  0.3045715 ,
        0.5592957 , -0.08467277, -0.39279073,  0.991757  ,  0.02293962,
       -0.35506874,  0.01170945,  0.74648327, -1.0782164 ,  0.0041171 ],
      dtype=float32)

In [8]:
type(df['Dairy free'][0])

numpy.int64

In [9]:
df.head()

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa...","[chicken, acorn, squash, sage, rosemary, butte...",22,1,0,0,0,1,0,1,1,"[-0.6576614, 0.08998758, -0.5539392, -0.238941..."
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"[2 large egg whites, 1 pound new potatoes (abo...","[egg, whites, new, potatoes, ground, black, pe...",7,0,1,0,0,0,0,0,1,"[-0.6207786, -0.18897343, -1.0001713, -0.42296..."
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"[1 cup evaporated milk, 1 cup whole milk, 1 ts...","[evaporated, milk, milk, garlic, powder, onion...",10,1,0,0,0,0,0,0,1,"[-0.553569, -0.8768758, -0.5323821, -0.2942076..."
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"[1 (¾- to 1-pound) round Italian loaf, cut int...","[round, italian, loaf, olive, oil, sweet, ital...",13,1,1,0,0,0,0,1,1,"[-0.35977286, -0.563806, -0.43339086, -0.18905..."
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"[1 teaspoon dark brown sugar, 1 teaspoon hot w...","[dark, brown, bourbon, fresh, lemon, juice, ap...",6,1,0,0,0,0,0,0,1,"[-0.73594683, 0.34916192, -0.23367356, 0.44848..."


In [10]:
df.shape

(13487, 16)

In [ ]:
['Vegetarian', 'Vegan', 'Gluten free', 'Nut free','No shellfish',
     'No eggs', 'Dairy free', 'No soy' ]

In [12]:
dr_1 = ['No soy']

In [13]:
df_multi = ['Gluten free', 'Dairy free']

In [25]:
dr_shortlist_df = filter_diet_req(df_multi, df)
dr_shortlist_df 

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List
0,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"[2 large egg whites, 1 pound new potatoes (abo...","[egg, whites, new, potatoes, ground, black, pe...",7,0,1,0,0,0,0,0,1,"[-0.6207786, -0.18897343, -1.0001713, -0.42296..."
1,Warm Comfort,"['2 chamomile tea bags', '1½ oz. reposado tequ...",Place 2 chamomile tea bags in a heatsafe vesse...,warm-comfort-tequila-chamomile-toddy,"[2 chamomile tea bags, 1½ oz. reposado tequila...","[chamomile, tea, bags, tequila, fresh, lemon, ...",4,0,0,0,0,0,0,1,1,"[0.5747835, 0.39186037, -0.09040865, 0.2512850..."
2,Turmeric Hot Toddy,"['¼ cup granulated sugar', '¾ tsp. ground turm...","For the turmeric syrup, combine ½ cup hot wate...",turmeric-hot-toddy-claire-sprouse,"[¼ cup granulated sugar, ¾ tsp. ground turmeri...","[granulated, ground, turmeric, amontillado, sh...",7,0,0,0,0,0,0,0,0,"[-0.02698247, 0.08067875, -0.18083556, 0.15382..."
3,Instant Pot Lamb Haleem,"['¾ cup assorted dals (such as chana dal, moon...","Combine dals, rice, and barley in a medium bow...",instant-pot-lamb-haleem,"[¾ cup assorted dals (such as chana dal, moong...","[assorted, dals, chana, dal, moong, dal, masoo...",17,0,0,0,0,0,0,1,1,"[-0.41612205, 0.4302757, -0.5840837, -0.364507..."
4,Spiral Ham in the Slow Cooker,"['1 (6- to 7-pound) bone-in, hickory-smoked, u...",Pour 3 cups of water into a 7-quart slow cooke...,spiral-ham-in-the-slow-cooker-guarnaschelli,"[1 (6- to 7-pound) bone-in, hickory-smoked, un...","[hickorysmoked, uncured, fully, ham, parsnips,...",10,0,0,0,0,0,0,1,1,"[-0.1755527, 0.055404864, -0.973434, 0.9869544..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5396,Red-Wine Vinaigrette,"['1/4 cup red-wine vinegar', '1 tablespoon Dij...","Whisk the vinegar, mustard, sugar, salt and pe...",red-wine-vinaigrette-106752,"[1/4 cup red-wine vinegar, 1 tablespoon Dijon ...","[redwine, vinegar, dijon, mustard, black, pepp...",6,0,0,0,0,0,0,0,0,"[-0.49512395, 0.10038737, -0.8658178, -0.43156..."
5397,Cider-Braised Pork Shoulder with Caramelized O...,['1 (3- to 4-lb) bone-in fresh pork shoulder h...,Preheat oven to 325°F.\nScore fat and any skin...,cider-braised-pork-shoulder-with-caramelized-o...,[1 (3- to 4-lb) bone-in fresh pork shoulder ha...,"[fresh, pork, shoulder, arm, picnic, garlic, c...",5,0,0,0,0,0,0,1,1,"[-0.77211696, 0.45787525, -0.66985196, -0.1214..."
5398,"Tomato, Garlic, and Potato Frittata","['6 whole large eggs', '2 large egg whites', '...","Whisk together whole eggs, whites, 1/4 cup par...",tomato-garlic-and-potato-frittata-105017,"[6 whole large eggs, 2 large egg whites, 1/2 c...","[eggs, egg, whites, parmesan, fresh, basil, bl...",10,0,1,0,0,0,0,0,1,"[-0.10650815, -0.38983095, -0.348332, -1.13059..."
5399,Dandelion Salad with Warm Bacon Dressing,"['1 lb tender dandelion greens, tough stems re...",Cut greens into 1 1/2-inch lengths and transfe...,dandelion-salad-with-warm-bacon-dressing-106375,"[1 lb tender dandelion greens, tough stems rem...","[dandelion, greens, bacon, shallot, cider, vin...",6,0,0,0,0,0,0,0,0,"[-0.25940752, 0.29297513, -0.8791934, 0.142414..."


In [39]:
!pwd

/Users/denisechan/code/denwychan/wots_cookin/notebooks


In [42]:
pd.read_csv("../ref_data/full_stopwords.csv")['stopwords']

0      bitesize
1         you'd
2       neutral
3            or
4           his
         ...   
572       curly
573      russet
574      maldon
575        tart
576        butt
Name: stopwords, Length: 577, dtype: object

In [82]:
stopwords = list(pd.read_csv("../ref_data/full_stopwords.csv")['stopwords'])
stopwords

['bitesize',
 "you'd",
 'neutral',
 'or',
 'his',
 'when',
 'bonein',
 'fillet',
 'water',
 'cm',
 'temperature',
 'minced',
 'approximately',
 'is',
 'yours',
 'pot',
 'on',
 'ripe',
 'both',
 'wedges',
 "it's",
 'so',
 'needn',
 'instant',
 'patted',
 'itself',
 'dusting',
 'see',
 'freshly',
 "wasn't",
 'percent',
 'which',
 'mightn',
 'outer',
 'extra',
 'serving',
 'extrafirm',
 'wellstirred',
 "wouldn't",
 'ice',
 'pitted',
 'ozg',
 'himself',
 'ounces',
 'until',
 'plus',
 'boneless',
 'couldn',
 'stock',
 'of',
 'finely',
 'cracked',
 'are',
 "doesn't",
 'cubes',
 'stems',
 'into',
 'slices',
 'oz',
 'rind',
 'desired',
 'clean',
 'thin',
 'boiling',
 'littleneck',
 'quartered',
 'loosely',
 'stemmed',
 'being',
 'down',
 'get',
 'crystal',
 'after',
 'but',
 'theirs',
 'were',
 'if',
 'tbsp',
 'divided',
 'an',
 'as',
 'steamed',
 'superfine',
 'lo',
 'had',
 'any',
 'meal',
 'during',
 'fl',
 'like',
 "she's",
 'only',
 'won',
 'sticks',
 'mortar',
 'mashed',
 'blend',
 'shee

In [45]:
transcript = 'I have in my fridge chicken aubergine celery garlic ginger'

In [112]:
ingredients = transcript.split()
ingredients = remove_stopwords_from_list(ingredients, stopwords)
ingredients

['chicken', 'aubergine', 'celery', 'garlic', 'ginger']

In [59]:
test_df = shortlist_df.copy()

In [96]:
list(test_df[test_df['Title'] == 'Bagels']['Bag_Of_Ingredients'])

[['g',
  'barley',
  'malt',
  'syrup',
  'honey',
  'rice',
  'syrup',
  'g',
  'diastatic',
  'malt',
  'powder',
  'g',
  'yeast',
  'g',
  'g',
  'lukewarm',
  'f',
  'c',
  'g',
  'bread',
  'flour',
  'g',
  'g',
  'barley',
  'malt',
  'syrup',
  'honey',
  'g',
  'baking',
  'soda',
  'g']]

In [120]:
new_df = shortlist_recipes(test_df, ingredients)
new_df

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List,Match_Score,Calcs,Ingredients_Available
1968,Spicy Chicken Stock,"['1 (3-pound) whole chicken', '1/2 bunch celer...","Place chicken, celery, onion, cilantro, carrot...",spicy-chicken-stock,"[1 (3-pound) whole chicken, 1/2 bunch celery, ...","[chicken, celery, onion, cilantro, parsley, ca...",13,0,0,0,0,0,0,1,1,"[-0.586363, 1.3950744, -1.0275725, -0.4037406,...",0.307692,Calculation: 4 / 13,4 / 13
199,Chicken Stock on the Grill,"['12 cups water', '8 garlic cloves', '2-inch p...",Wipe the grill grates with oil to prevent stic...,chicken-stock-on-the-grill,"[12 cups water, 8 garlic cloves, 2-inch piece ...","[garlic, cloves, fresh, ginger, star, anise, b...",13,0,0,0,0,0,0,1,1,"[-0.7388232, 0.9363986, -0.6343424, -0.3154486...",0.307692,Calculation: 4 / 13,4 / 13
1353,Gingery Chicken Soup with Zucchini “Noodles”,"['4 medium carrots (about 1 1/4 pounds), divid...","Scrub 2 carrots and cut into 2"" pieces. Finely...",gingery-chicken-soup-with-zucchini-noodles,"[4 medium carrots (about 1 1/4 pounds), divide...","[carrots, ginger, chicken, onion, sixths, cele...",13,0,0,0,0,0,0,1,1,"[-0.58671683, 0.69892246, -0.6055751, -0.32564...",0.307692,Calculation: 4 / 13,4 / 13
5558,Cream of Cauliflower Soup,"['2 leeks, including 2 inches of green, roots ...",1. Cut leeks in half lengthwise. Wash to remov...,cream-of-cauliflower-soup-350616,"[2 leeks, including 2 inches of green, roots t...","[leeks, including, inches, green, roots, olive...",13,1,0,0,0,0,0,1,1,"[-0.7015533, 0.30369675, -0.5280789, -0.324103...",0.307692,Calculation: 4 / 13,4 / 13
1889,Irish Stew with Pearl Barley,"['4 (12-oz., 1 1/4-inch–thick) bone-in lamb le...",Preheat the oven to 325°F.\nPlace a flameproof...,irish-lamb-stew-with-pearl-barley,"[4 (12-oz., 1 1/4-inch–thick) bone-in lamb leg...","[lamb, leg, chops, ground, black, pepper, carr...",10,0,0,0,0,0,0,1,1,"[-0.7745785, 0.25077882, -0.740841, -0.3772404...",0.300000,Calculation: 3 / 10,3 / 10


In [117]:
new_df = shortlist_recipes(test_df, ingredients)
new_df

compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <

compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <class 'list'>
compare me <

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List,Match_Score,Calcs,Ingredients_Available
1968,Spicy Chicken Stock,"['1 (3-pound) whole chicken', '1/2 bunch celer...","Place chicken, celery, onion, cilantro, carrot...",spicy-chicken-stock,"[1 (3-pound) whole chicken, 1/2 bunch celery, ...","[chicken, celery, onion, cilantro, parsley, ca...",13,0,0,0,0,0,0,1,1,"[-0.586363, 1.3950744, -1.0275725, -0.4037406,...",0.307692,Calculation: 4 / 13,Calculation: 4 / 13
1353,Gingery Chicken Soup with Zucchini “Noodles”,"['4 medium carrots (about 1 1/4 pounds), divid...","Scrub 2 carrots and cut into 2"" pieces. Finely...",gingery-chicken-soup-with-zucchini-noodles,"[4 medium carrots (about 1 1/4 pounds), divide...","[carrots, ginger, chicken, onion, sixths, cele...",13,0,0,0,0,0,0,1,1,"[-0.58671683, 0.69892246, -0.6055751, -0.32564...",0.307692,Calculation: 4 / 13,Calculation: 4 / 13
5558,Cream of Cauliflower Soup,"['2 leeks, including 2 inches of green, roots ...",1. Cut leeks in half lengthwise. Wash to remov...,cream-of-cauliflower-soup-350616,"[2 leeks, including 2 inches of green, roots t...","[leeks, including, inches, green, roots, olive...",13,1,0,0,0,0,0,1,1,"[-0.7015533, 0.30369675, -0.5280789, -0.324103...",0.307692,Calculation: 4 / 13,Calculation: 4 / 13
199,Chicken Stock on the Grill,"['12 cups water', '8 garlic cloves', '2-inch p...",Wipe the grill grates with oil to prevent stic...,chicken-stock-on-the-grill,"[12 cups water, 8 garlic cloves, 2-inch piece ...","[garlic, cloves, fresh, ginger, star, anise, b...",13,0,0,0,0,0,0,1,1,"[-0.7388232, 0.9363986, -0.6343424, -0.3154486...",0.307692,Calculation: 4 / 13,Calculation: 4 / 13
7192,Chicken Stir-Fry with Peanut Sauce Over Rice,"['Vegetable-oil cooking spray', '6 oz boneless...",Coat a nonstick pan with cooking spray; sauté ...,chicken-stir-fry-with-peanut-sauce-over-rice-2...,"[Vegetable-oil cooking spray, 6 oz boneless, s...","[vegetableoil, cooking, chicken, breast, sesam...",10,1,0,1,0,0,1,1,1,"[-0.5559505, 0.38789544, -0.67464554, -0.39522...",0.300000,Calculation: 3 / 10,Calculation: 3 / 10


In [104]:
new_df2 = shortlist_recipes2(test_df, transcript)
new_df2

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List,Match_Score
4133,Bagels,['1 tablespoon (0.75 oz / 21 g) barley malt sy...,"To make the dough, stir the malt syrup, yeast,...",bagels-366757,[1 tablespoon (0.75 oz / 21 g) barley malt syr...,"[g, barley, malt, syrup, honey, rice, syrup, g...",10,0,0,0,0,1,0,0,1,"[-0.5937052, -0.7301828, -0.6170337, 0.3037528...",1.000000
4089,Chicken & Rice Soup,['low-sodium chicken broth 4 cups (32 fl oz/1 ...,"Combine the broth, rice, and 2 cups (16 fl oz/...",chicken-rice-soup-366990,[low-sodium chicken broth 4 cups (32 fl oz/1 l...,"[chicken, broth, l, longgrain, white, rice, g,...",10,0,0,0,0,0,0,1,1,"[-0.1814429, 0.26556593, -0.48083973, -0.23907...",0.700000
5336,Shrimp and Andouille Pot Pies,['1 sheet frozen puff pastry (half of 16-ounce...,Preheat oven to 400°F. Roll out pastry on flou...,shrimp-and-andouille-pot-pies-351529,[1 sheet frozen puff pastry (half of 16-ounce ...,"[frozen, puff, pastry, packag, e, allbutter, p...",13,1,0,0,1,1,0,1,1,"[-0.13147235, -0.34613454, -0.5230868, -0.1963...",0.615385
507,Saffron Breakfast Kheer,"['1 cup (150 g) long-grain brown rice', '3/4 c...",Soak the rice in one bowl and the cashews and ...,saffron-breakfast-kheer,"[1 cup (150 g) long-grain brown rice, 3/4 cup ...","[g, longgrain, brown, rice, g, cashew, nuts, g...",12,0,0,1,0,0,0,0,1,"[-0.67862046, 0.050627824, -0.40478593, 0.1828...",0.583333
690,Meatballs and Noodle Soup (Almondigas),"['1 lb (450 g) lean ground pork', '2 green oni...","In a large bowl, combine the pork, green onion...",meatballs-and-noodle-soup-almondigas,"[1 lb (450 g) lean ground pork, 2 green onions...","[g, lean, ground, pork, green, onions, egg, g,...",13,0,1,0,0,1,0,1,1,"[-0.6751852, 0.032848164, -0.55999887, -0.3655...",0.538462


In [98]:
new_df

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List,Match_Score
4133,Bagels,['1 tablespoon (0.75 oz / 21 g) barley malt sy...,"To make the dough, stir the malt syrup, yeast,...",bagels-366757,[1 tablespoon (0.75 oz / 21 g) barley malt syr...,"[g, barley, malt, syrup, honey, rice, syrup, g...",10,0,0,0,0,1,0,0,1,"[-0.5937052, -0.7301828, -0.6170337, 0.3037528...",1.000000
4089,Chicken & Rice Soup,['low-sodium chicken broth 4 cups (32 fl oz/1 ...,"Combine the broth, rice, and 2 cups (16 fl oz/...",chicken-rice-soup-366990,[low-sodium chicken broth 4 cups (32 fl oz/1 l...,"[chicken, broth, l, longgrain, white, rice, g,...",10,0,0,0,0,0,0,1,1,"[-0.1814429, 0.26556593, -0.48083973, -0.23907...",0.700000
5336,Shrimp and Andouille Pot Pies,['1 sheet frozen puff pastry (half of 16-ounce...,Preheat oven to 400°F. Roll out pastry on flou...,shrimp-and-andouille-pot-pies-351529,[1 sheet frozen puff pastry (half of 16-ounce ...,"[frozen, puff, pastry, packag, e, allbutter, p...",13,1,0,0,1,1,0,1,1,"[-0.13147235, -0.34613454, -0.5230868, -0.1963...",0.615385
507,Saffron Breakfast Kheer,"['1 cup (150 g) long-grain brown rice', '3/4 c...",Soak the rice in one bowl and the cashews and ...,saffron-breakfast-kheer,"[1 cup (150 g) long-grain brown rice, 3/4 cup ...","[g, longgrain, brown, rice, g, cashew, nuts, g...",12,0,0,1,0,0,0,0,1,"[-0.67862046, 0.050627824, -0.40478593, 0.1828...",0.583333
690,Meatballs and Noodle Soup (Almondigas),"['1 lb (450 g) lean ground pork', '2 green oni...","In a large bowl, combine the pork, green onion...",meatballs-and-noodle-soup-almondigas,"[1 lb (450 g) lean ground pork, 2 green onions...","[g, lean, ground, pork, green, onions, egg, g,...",13,0,1,0,0,1,0,1,1,"[-0.6751852, 0.032848164, -0.55999887, -0.3655...",0.538462


In [86]:
new_df['Bag_Of_Ingredients'][0]

['g',
 'barley',
 'malt',
 'syrup',
 'honey',
 'rice',
 'syrup',
 'g',
 'diastatic',
 'malt',
 'powder',
 'g',
 'yeast',
 'g',
 'g',
 'lukewarm',
 'f',
 'c',
 'g',
 'bread',
 'flour',
 'g',
 'g',
 'barley',
 'malt',
 'syrup',
 'honey',
 'g',
 'baking',
 'soda',
 'g']

In [67]:
new_df = new_df[new_df['Match_Score']>0]

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List,Match_Score
5,Hot Pimento Cheese Dip,['1 (7 oz./200 g) can chipotle in adobo sauce'...,Put the chipotle peppers and adobo sauce in a ...,hot-pimento-cheese-dip-polina-chesnakova,"[1 (7 oz./200 g) can chipotle in adobo sauce, ...","[g, chipotle, adobo, sauce, garlic, clove, cor...",10,1,0,0,0,0,0,0,1,"[-0.14946051, -0.6622729, -0.36648086, -0.2985...",0.500000
14,Kale and Pumpkin Falafels With Pickled Carrot ...,['¾ cup (120g or 4½ oz) toasted pepitas (pumpk...,Preheat oven to 220°C (425°F).\nPlace the pepi...,kale-and-pumpkin-falafels-with-pickled-carrot-...,[¾ cup (120g or 4½ oz) toasted pepitas (pumpki...,"[g, pepitas, pumpkin, seeds, g, kale, leaves, ...",15,1,0,0,0,0,0,0,1,"[-0.40664175, 0.63175476, -0.7022262, 0.013121...",0.333333
15,Maple and Chile Roasted Squash With Quinoa Tab...,['1 (2kg or 4 lb or 4 oz) butternut squash or ...,Preheat oven to 220°C (425°F).\nScoop the seed...,maple-chile-roasted-pumpkin-with-quinoa-tabouli,[1 (2kg or 4 lb or 4 oz) butternut squash or 2...,"[butternut, squash, butternut, squash, virgin,...",14,1,0,1,0,0,0,0,1,"[-0.25190267, 0.12989262, -0.5606732, -0.20894...",0.357143
25,Buss Up Shut (Paratha Roti),"['2¼ tsp. nonfat dry milk powder', '2 tsp. bak...","Whisk milk powder, baking powder, granulated s...",buss-up-shut-paratha-roti-ramin-ganeshram,"[2¼ tsp. nonfat dry milk powder, 2 tsp. baking...","[nonfat, dry, milk, powder, baking, powder, gr...",10,1,0,0,0,1,0,0,1,"[-1.0159564, -0.7183108, -0.6062544, 0.1289316...",0.500000
28,Yogurt and Spice Roasted Salmon,"['500g skinless salmon fillet, cut into 4cm cu...",Preheat your oven to its highest setting (with...,easy-yogurt-and-spice-roasted-salmon-sabrina-g...,"[500g skinless salmon fillet, cut into 4cm cub...","[g, salmon, greek, yogurt, garlic, granules, h...",14,1,0,0,0,0,0,1,1,"[-0.2580709, 0.051951785, -0.5181383, -0.23258...",0.357143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7578,Maple Pumpkin Pie,"['2 recipes pastry dough', 'an egg wash made b...",On a lightly floured surface with a floured ro...,maple-pumpkin-pie-13429,"[2 recipes pastry dough, an egg wash made by b...","[recipes, pastry, dough, egg, wash, made, beat...",11,1,1,0,0,0,0,0,1,"[-0.29098117, -0.5481776, -0.3010396, 0.290190...",0.090909
7600,Ultimate Sticky Buns,"['1 cup warm water (105°F to 115°F)', '4 teasp...","Mix 1/4 cup warm water, yeast, and pinch of su...",ultimate-sticky-buns-106413,"[1 cup warm water (105°F to 115°F), 4 teaspoon...","[warm, f, f, dry, yeast, stick, butter, dry, n...",16,1,1,0,0,1,0,0,1,"[-0.86858255, -0.82306975, -0.59445703, 0.1223...",0.062500
7617,Drunken Chicken,['a 4- to 5-pound (about 2 kg) roasting chicke...,1. Wipe the chicken inside and out with paper ...,drunken-chicken-105465,[a 4- to 5-pound (about 2 kg) roasting chicken...,"[roasting, chicken, pepper, g, butter, sweet, ...",21,1,0,1,0,1,0,1,1,"[-0.38707194, -0.14956914, -0.60740924, -0.290...",0.238095
7655,Classic Terrine of Foie Gras,['1 (1 1/2-lb) whole raw Grade A duck or goose...,Preheat oven to 200°F and line a small roastin...,classic-terrine-of-foie-gras-104376,[1 (1 1/2-lb) whole raw Grade A duck or goose ...,"[raw, grade, duck, goose, foie, gras, ground, ...",10,0,0,0,0,0,0,1,1,"[-0.020728996, -0.21471176, -0.19247928, 0.064...",0.400000


In [63]:

new_df.sort_values(by = 'Match_Score', ascending=False, in)

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List,Match_Score
4133,Bagels,['1 tablespoon (0.75 oz / 21 g) barley malt sy...,"To make the dough, stir the malt syrup, yeast,...",bagels-366757,[1 tablespoon (0.75 oz / 21 g) barley malt syr...,"[g, barley, malt, syrup, honey, rice, syrup, g...",10,0,0,0,0,1,0,0,1,"[-0.5937052, -0.7301828, -0.6170337, 0.3037528...",1.000000
4089,Chicken & Rice Soup,['low-sodium chicken broth 4 cups (32 fl oz/1 ...,"Combine the broth, rice, and 2 cups (16 fl oz/...",chicken-rice-soup-366990,[low-sodium chicken broth 4 cups (32 fl oz/1 l...,"[chicken, broth, l, longgrain, white, rice, g,...",10,0,0,0,0,0,0,1,1,"[-0.1814429, 0.26556593, -0.48083973, -0.23907...",0.700000
5336,Shrimp and Andouille Pot Pies,['1 sheet frozen puff pastry (half of 16-ounce...,Preheat oven to 400°F. Roll out pastry on flou...,shrimp-and-andouille-pot-pies-351529,[1 sheet frozen puff pastry (half of 16-ounce ...,"[frozen, puff, pastry, packag, e, allbutter, p...",13,1,0,0,1,1,0,1,1,"[-0.13147235, -0.34613454, -0.5230868, -0.1963...",0.615385
507,Saffron Breakfast Kheer,"['1 cup (150 g) long-grain brown rice', '3/4 c...",Soak the rice in one bowl and the cashews and ...,saffron-breakfast-kheer,"[1 cup (150 g) long-grain brown rice, 3/4 cup ...","[g, longgrain, brown, rice, g, cashew, nuts, g...",12,0,0,1,0,0,0,0,1,"[-0.67862046, 0.050627824, -0.40478593, 0.1828...",0.583333
690,Meatballs and Noodle Soup (Almondigas),"['1 lb (450 g) lean ground pork', '2 green oni...","In a large bowl, combine the pork, green onion...",meatballs-and-noodle-soup-almondigas,"[1 lb (450 g) lean ground pork, 2 green onions...","[g, lean, ground, pork, green, onions, egg, g,...",13,0,1,0,0,1,0,1,1,"[-0.6751852, 0.032848164, -0.55999887, -0.3655...",0.538462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689,A Summer's Night Shrimp Cocktail,"['3 Roma tomatoes, halved lengthwise, plus 1/4...","Position a rack on the top level of the oven, ...",a-summers-night-shrimp-cocktail-51249430,"[3 Roma tomatoes, halved lengthwise, plus 1/4 ...","[roma, tomatoes, tomato, lime, juice, shrimp, ...",18,0,0,0,1,0,0,1,1,"[0.03307096, 0.4958821, -0.5270023, -0.5704868...",0.000000
2688,Guajillo-Braised Beef Short Rib Taco,"['8 guajillo chiles, stemmed and seeded', '3 d...",Preheat the oven to 325°F.\nWorking in two bat...,guajillo-braised-beef-short-rib-taco-51249220,"[8 guajillo chiles, stemmed and seeded, 3 drie...","[guajillo, chiles, dried, chipotle, chiles, ve...",18,0,0,0,0,0,0,1,1,"[-0.46379212, 0.58555675, -0.6737718, -0.60010...",0.000000
2687,"Tuna Tostadas, Contramar Style","['1 egg yolk, chilled', '1 tablespoon freshly ...","To make the aioli, combine the egg yolk, lemon...",tuna-tostadas-contramar-style-51249210,"[1 egg yolk, chilled, 1 tablespoon freshly squ...","[egg, yolk, lemon, juice, clove, garlic, veget...",15,0,1,0,0,0,1,1,1,"[-0.122210875, 0.21654582, -0.53289187, -0.348...",0.000000
2686,"Butternut Squash, Kale, and Crunchy Pepitas Taco","['2/3 cup raw cashews', '1 teaspoon cumin seed...","To make the crema, soak the raw cashews in roo...",butternut-squash-kale-and-crunchy-pepitas-taco...,"[2/3 cup raw cashews, 1 teaspoon cumin seeds, ...","[raw, cashews, cumin, seeds, lime, juice, lime...",21,1,0,1,0,0,0,0,1,"[-0.30625272, 0.5189675, -0.76835865, -0.27903...",0.000000


In [27]:
shortlist_df = filter_min_ingredients(10, df)
shortlist_df

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa...","[chicken, acorn, squash, sage, rosemary, butte...",22,1,0,0,0,1,0,1,1,"[-0.6576614, 0.08998758, -0.5539392, -0.238941..."
1,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"[1 cup evaporated milk, 1 cup whole milk, 1 ts...","[evaporated, milk, milk, garlic, powder, onion...",10,1,0,0,0,0,0,0,1,"[-0.553569, -0.8768758, -0.5323821, -0.2942076..."
2,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"[1 (¾- to 1-pound) round Italian loaf, cut int...","[round, italian, loaf, olive, oil, sweet, ital...",13,1,1,0,0,0,0,1,1,"[-0.35977286, -0.563806, -0.43339086, -0.18905..."
3,Instant Pot Lamb Haleem,"['¾ cup assorted dals (such as chana dal, moon...","Combine dals, rice, and barley in a medium bow...",instant-pot-lamb-haleem,"[¾ cup assorted dals (such as chana dal, moong...","[assorted, dals, chana, dal, moong, dal, masoo...",17,0,0,0,0,0,0,1,1,"[-0.41612205, 0.4302757, -0.5840837, -0.364507..."
4,Spiced Lentil and Caramelized Onion Baked Eggs,"['1 (14.5-ounce) can basic lentil soup, like A...","Place an oven rack in the center of the oven, ...",spiced-lentil-and-caramelized-onion-baked-eggs,"[1 (14.5-ounce) can basic lentil soup, like Am...","[basic, lentil, soup, amys, onion, turmeric, c...",13,1,1,0,0,0,0,0,1,"[-0.6637677, -0.029937275, -0.67289865, -0.584..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7735,Chocolate Heart Layer Cake with Chocolate-Cinn...,"['4 tablespoons unsalted butter, melted', '1/3...",Preheat oven to 400°F. Place 8x8x2-inch heart-...,chocolate-heart-layer-cake-with-chocolate-cinn...,"[4 tablespoons unsalted butter, melted, 1/3 cu...","[butter, flour, cocoa, powder, eggs, heavy, wh...",17,1,1,0,0,1,0,0,1,"[-0.38265464, -0.6943872, -0.300229, -0.030650..."
7736,Ginger-Pecan Roulade with Honey-Glazed Pecans,"['1/2 stick (1/4 cup) unsalted butter, melted,...",Preheat oven to 350°F. Line bottom and sides o...,ginger-pecan-roulade-with-honey-glazed-pecans-...,"[1/2 stick (1/4 cup) unsalted butter, melted, ...","[stick, butter, pecans, cake, flour, selfrisin...",15,1,1,0,0,1,0,0,1,"[-1.1408284, -0.28580016, -0.38751838, 0.35382..."
7737,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"[1 cup all-purpose flour, 2/3 cup unsweetened ...","[allpurpose, flour, cocoa, powder, doubleactin...",13,1,1,1,0,1,0,0,1,"[-1.0437251, -0.9660939, -0.4199381, 0.3009258..."
7738,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"[1 preserved lemon, 1 1/2 pound butternut squa...","[preserved, lemon, butternut, squash, olive, o...",10,1,0,1,0,1,0,0,1,"[-0.4856219, 0.093395606, -0.30903456, -0.1750..."


In [11]:
boi = df['Bag_Of_Ingredients']
boi

0        [chicken, acorn, squash, sage, rosemary, butte...
1        [egg, whites, new, potatoes, ground, black, pe...
2        [evaporated, milk, milk, garlic, powder, onion...
3        [round, italian, loaf, olive, oil, sweet, ital...
4        [dark, brown, bourbon, fresh, lemon, juice, ap...
                               ...                        
13482    [allpurpose, flour, cocoa, powder, doubleactin...
13483    [preserved, lemon, butternut, squash, olive, o...
13484    [katsuo, bushi, dried, bonito, making, dashi, ...
13485    [stick, butter, baby, spinach, feta, nutmeg, p...
13486    [fresh, poblano, chiles, tomatoes, including, ...
Name: Bag_Of_Ingredients, Length: 13487, dtype: object

In [17]:
model = Trainer()

In [19]:
model.train_model(boi)

Recipes corpus set
13487 recipes vectorized
Model trained!
Model saved!


In [21]:
model.__dict__

{'words': ['fresh',
  'oil',
  'ground',
  'pepper',
  'olive',
  'butter',
  'garlic',
  'juice',
  'black',
  'lemon',
  'red',
  'flour',
  'leaves',
  'white',
  'cloves',
  'cream',
  'vinegar',
  'vegetable',
  'onion',
  'dried',
  'extravirgin',
  'powder',
  'seeds',
  'chicken',
  'sauce',
  'vanilla',
  'egg',
  'eggs',
  'cheese',
  'wine',
  'allpurpose',
  'baking',
  'milk',
  'stick',
  'green',
  'lime',
  'parsley',
  'g',
  'extract',
  'dry',
  'brown',
  'tomatoes',
  'orange',
  'ginger',
  'heavy',
  'rice',
  'cilantro',
  'thyme',
  'cinnamon',
  'broth',
  'chocolate',
  'onions',
  'mustard',
  'frozen',
  'light',
  'bread',
  'honey',
  'clove',
  'syrup',
  'coconut',
  'mint',
  'basil',
  'celery',
  'beans',
  'potatoes',
  'sweet',
  'corn',
  'chiles',
  'cumin',
  'dark',
  'granulated',
  'soda',
  'sesame',
  'soy',
  'parmesan',
  'italian',
  'paste',
  'baby',
  'chile',
  'scallions',
  'bay',
  'yellow',
  'carrots',
  'flatleaf',
  'mushrooms

In [38]:
model = Word2Vec.load("../ref_data/word2vec.model")

In [55]:
model.__dict__['vector_size']

50

In [49]:
model.wv["apple"]

array([-0.9490214 ,  2.0655496 , -0.24119185,  3.185231  , -1.6865894 ,
        1.4353591 ,  2.093874  , -0.36742866, -1.30915   ,  0.3519775 ,
       -0.26778355, -2.5977676 ,  1.0187938 ,  0.93439007,  0.52544206,
        0.69450516, -0.6307321 ,  0.18665496, -1.9147391 ,  0.7997327 ,
       -1.8808897 ,  2.5445423 ,  1.5751626 ,  0.07857831,  0.67576516,
       -0.8177158 ,  1.2120743 , -1.2468091 , -0.42903134, -2.1660492 ,
       -0.6025046 , -0.52207184, -0.27904594,  0.5714336 , -1.2856464 ,
        1.0836418 , -0.13497362, -1.0256466 ,  0.08188655, -0.27139518,
       -1.4083208 , -1.2611444 ,  0.18512613,  0.88063604, -1.2329047 ,
       -0.67805886, -0.91457176,  0.9171641 , -2.0615797 , -2.2157543 ],
      dtype=float32)

In [60]:
ing_vector = get_ingredients_vector(model, ingredients)

In [61]:
ing_vector

array([-0.5419769 ,  0.19191222, -0.9757336 , -0.752201  ,  0.21766107,
       -1.1072496 ,  0.2964505 ,  0.3987936 , -0.8400328 , -1.41879   ,
        0.19042319, -0.8639715 ,  0.3689083 ,  0.20666893,  1.377828  ,
        0.0235855 ,  0.79750055, -0.84216315, -0.1882792 , -0.01432967,
       -0.77906424,  0.02414175,  1.9354633 , -0.07205006,  0.24045213,
       -1.3171417 , -0.8311513 ,  1.1269228 ,  0.19624878, -0.1872536 ,
        0.8199528 , -0.3035871 ,  0.26550493, -1.2584059 ,  0.05398284,
        1.9898347 ,  0.23988646, -0.19640428, -0.24728458,  0.7856892 ,
        0.44797635, -0.2749671 ,  0.2564443 ,  0.33795765,  0.2909512 ,
       -1.0065283 ,  0.3108195 ,  1.3412343 , -1.9398689 , -0.15940683],
      dtype=float32)

In [64]:
get_similar_recipes(ing_vector, df['Vector_List'], df)

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,Bag_Of_Ingredients,Ingredients_Length,Dairy free,No eggs,Nut free,No shellfish,Gluten free,No soy,Vegetarian,Vegan,Vector_List
11149,Smoked Turkey and Bacon Chowder with Pumpernic...,"['2 tablespoons vegetable oil', '4 smoked turk...",Heat oil in large pot over medium-high heat. A...,smoked-turkey-and-bacon-chowder-with-pumpernic...,"[2 tablespoons vegetable oil, 4 smoked turkey ...","[vegetable, oil, smoked, turkey, drumsticks, a...",15,0,0,0,0,0,0,1,1,"[-0.3382281, -0.19241378, -0.5166122, -0.35740..."
4064,Beef Stew in the Crock Pot,"['2 pounds chuck meat, cut into 1 1/2-inch cub...",Place the cubed meat and flour and in Ziploc b...,beef-stew-in-the-crock-pot-51262880,"[2 pounds chuck meat, cut into 1 1/2-inch cube...","[chuck, meat, flour, olive, oil, red, wine, be...",14,0,0,0,0,1,0,1,1,"[-0.6531993, 0.28295743, -0.8477734, -0.817279..."
10977,Herb-Roasted Leg of Lamb with Vegetables and Jus,"['4 medium onions, peeled (roots trimmed but s...","Preheat oven to 400°F.\nIn roasting pan, combi...",herb-roasted-leg-of-lamb-with-vegetables-and-j...,"[4 medium onions, peeled (roots trimmed but st...","[onions, roots, still, stalk, celery, garlic, ...",12,0,0,0,0,0,0,1,1,"[-0.7548514, 0.2489509, -0.5852069, -0.5964880..."
9687,Lentil Soup with Italian Sausage and Escarole,"['1 2/3 cups lentils (11 ounces), rinsed well'...","Simmer lentils, water, broth, bay leaf, and ha...",lentil-soup-with-italian-sausage-and-escarole-...,"[1 2/3 cups lentils (11 ounces), rinsed well, ...","[lentils, well, reducedsodium, chicken, broth,...",14,0,0,0,0,0,0,1,1,"[-0.33534053, 0.16587779, -0.56106627, -0.4904..."
4004,Double-Dark Chicken Noodle Soup,"['1 leek', 'Stems from 1/2 bunch parsley', '6 ...",Trim dark green leaves from leek; discard all ...,double-dark-chicken-noodle-soup-51264240,"[1 leek, Stems from 1/2 bunch parsley, 6 sprig...","[leek, parsley, thyme, bay, leaves, vegetable,...",19,0,1,0,0,1,0,1,1,"[-0.6511402, 0.33469975, -0.5782379, -0.422323..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Chopped BLT Salad,"['8 bacon slices, cooked and diced (about 1/2 ...","In a large bowl, combine all ingredients.\nSer...",chopped-blt-salad,"[8 bacon slices, cooked and diced (about 1/2 c...","[bacon, purple, cabbage, hearts, romaine, onio...",8,0,0,0,0,0,0,0,0,"[-0.27927768, 0.040377367, -0.5183899, -0.4774..."
12194,Giblet and Mushroom Bread Salad,"['Giblets (liver, neck, heart, and gizzard) fr...",Chop liver and transfer to small bowl; cover a...,giblet-and-mushroom-bread-salad-233156,"[Giblets (liver, neck, heart, and gizzard) fro...","[giblets, liver, neck, heart, gizzard, turkey,...",12,0,0,0,0,1,0,1,1,"[-0.19465132, 0.027061712, -0.59321606, -0.442..."
9046,Corned Beef with Cabbage,"['4 lb corned brisket of beef', '3 large carro...",Put the brisket into a saucepan with the carro...,corned-beef-with-cabbage-241775,"[4 lb corned brisket of beef, 3 large carrots,...","[corned, brisket, beef, carrots, onions, dry, ...",7,0,0,0,0,0,0,1,1,"[-0.4360143, 0.2490592, -0.66204214, -0.150502..."
6539,Chicken and Sausage Gumbo,"['1 tablespoon kosher salt plus more', '1 teas...",Combine 1 tablespoon salt and next 3 ingredien...,chicken-and-sausage-gumbo-380586,"[1 tablespoon kosher salt plus more, 1 teaspoo...","[ground, black, pepper, paprika, cayenne, pepp...",21,0,0,0,0,1,0,1,1,"[-0.40668535, 0.08093262, -0.7014194, -0.39020..."


In [128]:
ingredient_str = 'In my fridge I have Two chickens four Celery two onions ginger garlic'
ingredient_str

'In my fridge I have Two chickens four Celery two onions ginger garlic'

In [126]:
test2 = ingredient_str.split()
test2

['In',
 'my',
 'fridge',
 'I',
 'have',
 'Two',
 'chickens',
 'four',
 'Celery',
 'two',
 'onions',
 'ginger',
 'garlic']

In [116]:
stopwords = load_full_stopwords()

Returning list of 461 stopwords


In [136]:
stopwords

['bitesize',
 "you'd",
 'neutral',
 'or',
 'his',
 'when',
 'bonein',
 'fillet',
 'water',
 'cm',
 'temperature',
 'minced',
 'approximately',
 'is',
 'yours',
 'pot',
 'on',
 'ripe',
 'both',
 'wedges',
 "it's",
 'so',
 'needn',
 'instant',
 'patted',
 'itself',
 'dusting',
 'see',
 'freshly',
 "wasn't",
 'percent',
 'which',
 'mightn',
 'outer',
 'extra',
 'serving',
 'extrafirm',
 'wellstirred',
 "wouldn't",
 'ice',
 'pitted',
 'ozg',
 'himself',
 'ounces',
 'until',
 'plus',
 'boneless',
 'couldn',
 'stock',
 'of',
 'finely',
 'cracked',
 'are',
 "doesn't",
 'cubes',
 'stems',
 'into',
 'slices',
 'oz',
 'rind',
 'desired',
 'clean',
 'thin',
 'boiling',
 'littleneck',
 'quartered',
 'loosely',
 'stemmed',
 'being',
 'down',
 'get',
 'crystal',
 'after',
 'but',
 'theirs',
 'were',
 'if',
 'tbsp',
 'divided',
 'an',
 'as',
 'steamed',
 'superfine',
 'lo',
 'had',
 'any',
 'meal',
 'during',
 'fl',
 'like',
 "she's",
 'only',
 'won',
 'sticks',
 'mortar',
 'mashed',
 'blend',
 'shee

In [140]:
test2 = ingredient_str.split()
# ingredient_list = [w.lower() for w in test2 if not w.lower() in stopwords]
ingredient_list = remove_stopwords_from_list(test2, stopwords)
ingredient_list

['fridge',
 'two',
 'chickens',
 'four',
 'celery',
 'two',
 'onions',
 'ginger',
 'garlic']

In [94]:
ing_vector_test2 = get_ingredients_vector(model, test2)
ing_vector_test2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [106]:
model.wv.__dict__.keys()
model.wv.__dict__['index_to_key'][:10]

['fresh',
 'oil',
 'ground',
 'pepper',
 'olive',
 'butter',
 'garlic',
 'juice',
 'black',
 'lemon']

In [12]:
type(df['Vector_List'][0])

numpy.ndarray

Scoring - matches with words in bag of words and ingredients in cleaned ingredients

In [ ]:
model = Word2Vec.load("../ref_data/word2vec.model")

ingredient_str.split()
#remove stopwords 
get_ingredients_vector(model, ingredients)
shortlist = get_similar_recipes(ing_vector, df['Vector_List'], df)

In [ ]:
# Create slider to allow the user to select the number of results
# in the results dataframe
results_count = st.slider('Select number of results', 1, 15, 5)

# and used in order to select the displayed lines
results_df = results_df.head(results_count)

results_df

In [ ]:
more stop words


'fridge',
'two',
'four',